In [1]:
import sys 
sys.path.append('/host/d/Github/')
import os
import torch
import numpy as np
import nibabel as nb
import random
import matplotlib.pyplot as plt

import torch
from torch import nn, einsum
import torch.nn.functional as F

from tqdm import tqdm
from einops import rearrange, repeat, reduce
from scipy.ndimage import zoom

import CT_registration_diffusion.functions_collection as ff
import CT_registration_diffusion.Build_lists.Build_list as Build_list
import CT_registration_diffusion.Data_processing as Data_processing


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# first let's load a case
main_path = '/host/d/Data/4DCT'
dataset = 'DIR_LAB'
case_id = 'Case1'

img_path = os.path.join(main_path, dataset, case_id, 'cropped_image', 'img_0.nii.gz')
img = nb.load(img_path).get_fdata()

# put into cuda
img_tensor = torch.from_numpy(img).unsqueeze(0).unsqueeze(0).float().cuda()
# make sure now the dimension is [batch, channel, x,y,z]
print('image tensor shape:', img_tensor.shape)


image tensor shape: torch.Size([1, 1, 224, 224, 96])


to luxin: 参考function:

In [ ]:
def warp_segmentation_from_mvf(seg_t0, mvf_voxel):
    """
    seg_t0:    [B, 1, X, Y, Z] → will be permuted to [B, 1, Z, Y,X]
    mvf_voxel: [B, 3, X, Y, Z] → will be permuted to [B, 3, Z, Y,X]
    returns:
        warped_seg: [B, 1, X, Y, Z]
    """
    # Step 0: permute to [B, 1, Z, Y,X]
    seg_t0 = seg_t0.permute(0, 1, 4, 3, 2).contiguous()
    mvf_voxel = mvf_voxel.permute(0, 1, 4, 3, 2).contiguous()

    B, _, D, H, W = seg_t0.shape  # Z, Y, X
    device = seg_t0.device

    # Step 1: normalize MVF to [-1, 1]
    mvf_norm = torch.zeros_like(mvf_voxel)
    mvf_norm[:, 0] = mvf_voxel[:, 0] * 2 / (W - 1)  # dx
    mvf_norm[:, 1] = mvf_voxel[:, 1] * 2 / (H - 1)  # dy
    mvf_norm[:, 2] = mvf_voxel[:, 2] * 2 / (D - 1)  # dz

    # Step 2: create identity grid in Z, Y, X order
    grid_z = torch.linspace(-1, 1, D, device=device)
    grid_y = torch.linspace(-1, 1, H, device=device)
    grid_x = torch.linspace(-1, 1, W, device=device)
    meshz, meshy, meshx = torch.meshgrid(grid_z, grid_y, grid_x, indexing='ij')  # [D, H, W]
    identity_grid = torch.stack((meshx, meshy, meshz), dim=-1)  # [D, H, W, 3]
    identity_grid = identity_grid.unsqueeze(0).repeat(B, 1, 1, 1, 1)  # [B, D, H, W, 3]

    # Step 3: add displacement
    displacement_grid = identity_grid + mvf_norm.permute(0, 2, 3, 4, 1)  # [B, D, H, W, 3]

    # Step 4: warp
    warped = F.grid_sample(
        seg_t0, displacement_grid,
        mode='bilinear', padding_mode='border', align_corners=True
    )  # [B, 1, D, H, W]

    # Step 5: permute back to [B, 1, X, Y, Z]
    warped = warped.permute(0, 1, 4,3, 2).contiguous()

    return warped  # [B, 1, X, Y, Z]

In [ ]:
#### use or change the above function to write spatial transform module, that can be applied to img_tensor

# can you use your code to (1) translate? (2) rotate? 
# you can ask GPT: given the above function, if I want to translate the image by 5 voxels in x direction, how should I set mvf_voxel?